In [18]:
# load tidy dataset

import pandas as pd
import os.path
import numpy as np
from itertools import combinations
import glob
import sys
import time
from sklearn import svm
from sklearn.model_selection import train_test_split
import pdb

## to increase the cell width of the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# from sklearn.learning_curve import learning_curve
# from sklearn.grid_search import GridSearchCV
# from sklearn.cross_validation import ShuffleSplit

def loadTidyTimings(tidyTimingsCSV): 
    assert os.path.isfile(tidyTimingsCSV), "desired file does not exist" 
    df = pd.read_csv(tidyTimingsCSV, header = None, names = ["animal", "session", "trial", "stimulus", "onsetFrame", "offsetFrame", "onsetTime", "offsetTime"])
    return df 

def loadTidyCalcium(tidyData): 
    assert os.path.isfile(tidyData), "desired file does not exist" 
    df = pd.read_csv(tidyData, header = None, names = ["date", "animal", "session", "trial", "stimulus", "neuronID", "timePt", "CaSignal"])
    return df 

# helper functions to return the number of distinct types in the provided data frame
getNumNeurons = lambda df: len(np.unique(df['neuronID'].tolist()))
getNeurons = lambda df: np.unique(df['neuronID'].tolist())

getNumTrials = lambda df: len(np.unique(df['trial'].tolist()))
getTrials = lambda df: np.unique(df['trial'].tolist())

getOnsetFrameNum = lambda animal, session, trial: df_timings.loc[(df_timings['animal']==animal) & (df_timings['session']==session) & (df_timings['trial']==trial), ['onsetFrame']].values[0][0]
getOffsetFrameNum = lambda animal, session, trial: df_timings.loc[(df_timings['animal']==animal) & (df_timings['session']==session) & (df_timings['trial']==trial), ['offsetFrame']].values[0][0]


# pass in pre-filtered data set containing data for only one animal and the same session (ie SAME NEURONS!)
def getListsOfTrialIDs(df_animalSession):
    ## get trials for both stimuli
    df_anmlSessStimA = df_animalSession[df_animalSession['stimulus'] == stimA]
    df_anmlSessStimB = df_animalSession[df_animalSession['stimulus'] == stimB]
    print(stimA,stimB)

    ## get lists of trial numbers of each stimuli's presentations 
    trials_stimA = np.unique(df_anmlSessStimA['trial'].tolist())
    trials_stimB = np.unique(df_anmlSessStimB['trial'].tolist())
    print("trial IDs for each stimulus type",trials_stimA,trials_stimB)
    return (trials_stimA,trials_stimB)

## pass in a data frame with only a single animal and session    
def getNumTimePtsPerTrial(df_animalSession, trials_stimA, trials_stimB):

    #### get number of timePts in each trial selected above 
    ## (1 to 3 presentations of the same stimuli exist per session in Prabhat's data)
    numTimePtsPerTrial = np.empty((2,max(len(trials_stimA),len(trials_stimB))))
    numTimePtsPerTrial[:] = np.nan
    stimInd = 0;
    for thisStimTypeTrialNums in [trials_stimA, trials_stimB]:
        trialInd = 0
        for trial in thisStimTypeTrialNums:
            inds_thisTrial = (df_animalSession['trial']==trial)
            tmp_df_thisTrial = df_animalSession[inds_thisTrial] # gives all time points for all neurons
            numNeurons = getNumNeurons(tmp_df_thisTrial) 
            numTimePtsPerTrial[stimInd,trialInd] = np.sum(inds_thisTrial)/numNeurons 
            trialInd += 1
        stimInd += 1
    print('stimuliType x presentationNum (row x col)')    
    print(numTimePtsPerTrial) # rows are for stimuli type; cols are presentation of that stimulus
    return numTimePtsPerTrial

## test candidate comparisons based on whether the number of trials per session and approximate number of timePts match
def areNumTrialsPerStimulusEqual(numTimePtsPerTrial):
    
    ## no trials of either type --> discard this comparison for this animal/session   
    if np.all(np.isnan(numTimePtsPerTrial)):
        print("DISCARDED: neither stimulus type were found for this animal and session")
        return False  # skip to next session (WORK: handle this)
        
    ## different numbers of trials per stimuli/session --> discard this comparison for this animal/session 
    elif np.any(np.isnan(numTimePtsPerTrial)): 
        print("DISCARDED: mismatching numbers of trials per stimulus type for this animal/session")
        return False # skip to next session (WORK: handle this)

    ## FULFILLED here: condition that allows analysis to proceed to attempted data
    elif not np.any(np.isnan(numTimePtsPerTrial)): 
        print("trial numbers match")
    else:
        raise RuntimeError('unexpected trial comparison occurred')
        return False
    
    print("checking approx num of time points")

## input argument generated from getNumTimePtsPerTrial
def areNumTimePtsPerTrialSimilar(numTimePtsPerTrial):
    if np.isnan(threshTPs_stdFromMean) == True:
        return True
    minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)             
    if (np.abs(minTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))) \
        or (np.abs(maxTPs-meanTPs) > (threshTPs_stdFromMean * np.abs(meanTPs-stdTPs))): 
        print("DISCARDED: variance in trial length is above the user's threshold")
        return False # skip to next session (WORK: handle this)

    ### passed all criteria if it made it this far
    return True

## input argument created by getNumTimePtsPerTrial function
def timePtStats(numTimePtsPerTrial):
    minTPs = int(np.amin(numTimePtsPerTrial))
    maxTPs = int(np.amax(numTimePtsPerTrial))
    meanTPs = np.mean(numTimePtsPerTrial)
    stdTPs = np.std(numTimePtsPerTrial)

    ## useful for debugging
#     print('min', minTPs)
#     print('max', maxTPs)
#     print('std', stdTPs)
#     print('mean',meanTPs)
#     print('|min-mean|=',np.abs(minTPs-meanTPs))
#     print('|max-mean|=',np.abs(maxTPs-meanTPs))
#     print('|mean-std|=',np.abs(meanTPs-stdTPs))
#     print('thresh * |mean-std|=',(threshTPs_stdFromMean * np.abs(meanTPs-stdTPs)))
    
    return minTPs, maxTPs, meanTPs, stdTPs

def sameNeuronConcat(df_trunc, truncFrameNum):
    neuronArr_anmlSess_stimA = np.full((getNumNeurons(df_trunc),truncFrameNum-1), np.nan) # -1 for 0 indexing
    neuronArr_anmlSess_stimB = np.full((getNumNeurons(df_trunc),truncFrameNum-1), np.nan) # -1 for 0 indexing
    for stimLst in [trials_stimA, trials_stimB]:
        if np.array_equal(stimLst,trials_stimA) == True:
            print('\nstimulus:', stimA)
        elif np.array_equal(stimLst,trials_stimB) == True:
            print('\nstimulus:', stimB)
        else:
            raise RuntimeError('unexpected trial concatenation condition occurred')
        print('truncation frame: ', truncFrameNum)

        ## create temporary sub matrix of concatenated cells for ONE stimulus
        tmp_neuronsArr_sameStim = np.empty((getNumNeurons(df_trunc),truncFrameNum)) # no -1 DESPITE 0 indexing
        for trial in stimLst:
            print("appending same neurons in trial: ", trial)

            ##  create temporary sub matrix of same trial all cells
            tmp_neuronsArr_sameStim_sameTrial = np.empty((getNumNeurons(df_trunc),truncFrameNum)) # no -1 DESPITE 0 indexing
            for neuron in getNeurons(df_trunc):
                tmp_neuronInds = (df_trunc['trial']==trial) & (df_trunc['neuronID']==neuron)
                tmp_neuronSeries = df_trunc.loc[tmp_neuronInds,'CaSignal']
                
                ## pandas to numpy conversion
                tmp_neuronVec = tmp_neuronSeries.as_matrix()
                tmp_neuronsArr_sameStim_sameTrial[neuron-1,:] = tmp_neuronVec  
#                 print('neuronVec',np.shape(tmp_neuronVec))
#                 print('neuronsArr_sameStim_sameTrial', np.shape(tmp_neuronsArr_sameStim_sameTrial))
                 
            ## append trials to right of same stim if not the first entry
            if trial == stimLst[0]:
                tmp_neuronsArr_sameStim = np.copy(tmp_neuronsArr_sameStim_sameTrial)
            else:    
                tmp_neuronsArr_sameStim = np.concatenate((tmp_neuronsArr_sameStim, tmp_neuronsArr_sameStim_sameTrial), axis=1)
            print('same stim:', np.shape(tmp_neuronsArr_sameStim))

        ## save concatenated data to output variables
        if np.array_equal(stimLst,trials_stimA):
            neuronArr_anmlSess_stimA = tmp_neuronsArr_sameStim
        elif np.array_equal(stimLst,trials_stimB):
            neuronArr_anmlSess_stimB = tmp_neuronsArr_sameStim
        else:
            raise RuntimeError('unexpected same neuron concatenation state occured')
        
    return neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB

In [2]:
################ concatenate all .csv files exported from matlab into single pandas dataframe df

### tidy csv file and dir (use makeTidy_Anderson.m to convert Ann's structure to csv)

## for Ubuntu laptop
tidyDataDir = '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/' 
tidyTimingsFileAndPath = '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/stimulusTimings.csv' # for Ubuntu laptop

## for Anderson desktop
# tidyDataDir = 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\data\\2_tidyCSVformat\\' 
# tidyTimingsFileAndPath = 'C:\\Users\\public.Analysis\\Desktop\\MHR\\git_ignored\\data\\2_tidyCSVformat\\stimulusTimings.csv' # for Anderson desktop

tidyDataFileTemplate = 'mouse'
tidyDataFileExt = '.csv'

dataTemplate = tidyDataDir + tidyDataFileTemplate + '*' + tidyDataFileExt 


print("loading stimulus timings into df_timings")
timingsLst = []
print(tidyTimingsFileAndPath)
timingsLst.append(loadTidyTimings(tidyTimingsFileAndPath))
df_timings = pd.concat(timingsLst)

# get all input files you want to add to the same dataset
dataFiles = np.sort(glob.glob(dataTemplate))
print("\n data files: \n", dataFiles)

print("\n loading and appending to prior pandas data frame")
dataLst = []

for file in dataFiles:
    print(file)
    dataLst.append(loadTidyCalcium(file))
df = pd.concat(dataLst)

print('finished loading')

############# ALL DATA STORED HERE IN DF
    

loading stimulus timings into df_timings
/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/stimulusTimings.csv

 data files: 
 [ '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse1.csv'
 '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse2.csv'
 '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse3.csv'
 '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse4.csv'
 '/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse5.csv']

 loading and appending to prior pandas data frame
/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tidyCSVformat/mouse1.csv
/home/orthogonull/a_MHR/aa_research/aa_gitResearch/git_ignored/imagingAnalysis/data/2_tid

In [3]:
## survey/search data to prepare for split operation
metaStrs = [['dates','date'],['animals','animal'],['sessions','session'],['maxTrials','trial'],['stimuli','stimulus']]

print("searching over entire data set to get range of various IDs for data (used in subsequent loops)") 

## this dictionary holds useful info regarding the range of inputs to loop/search over subsequently
metaDct = {}
for a,b in metaStrs:
    print(a,b)
    metaDct[a] = np.unique(df[b].tolist())
print(metaDct)


searching over entire data set to get range of various IDs for data (used in subsequent loops)
dates date
animals animal
sessions session
maxTrials trial
stimuli stimulus
{'dates': array(['2017_05_00'], 
      dtype='<U10'), 'animals': array([1, 3, 4, 5, 7]), 'sessions': array([1, 2, 3]), 'maxTrials': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21]), 'stimuli': array(['USS', 'baseline', 'female', 'male', 'mineral oil odor',
       'peanut odor', 'pred odor', 'rat', 'tone', 'toy'], 
      dtype='<U16')}


In [21]:
######### USER PARAMETERS #########
threshTPs_stdFromMean = np.nan ## set to np.isnan if you don't want to filter based on number of TPs WORK: make this std of each type and not all types

In [22]:
# get all pairs of stimuli
stimCmbTpl = tuple(combinations(metaDct['stimuli'],2)) 


######### MAIN LOOP ##########
df_SVM = pd.DataFrame(columns=('dateOfAnalysis', 'dateOfExperiment', 'animal', 'session', 'stimulusA', 'stimulusB', 'SVM_accuracy'))
ind_comparison = 0;
for (stimA, stimB) in stimCmbTpl:
    print((stimA,stimB))
    
    ## get all data for both trial types
    indsBoth = (df['stimulus']==stimA) | (df['stimulus']==stimB)
    df_bothStimuli = df[indsBoth]

    #### select data by animals and sessions
    for animal in metaDct['animals']:
        print("stimuli comparison num: ", ind_comparison+1)
        print('animal: ', animal)
        for session in metaDct['sessions']:
            print('session:', session)
            
            ## return subselection of data where the same neurons were recorded
            inds_animalSession = (df_bothStimuli['animal'] == animal) & (df_bothStimuli['session'] == session)
            df_animalSession = df_bothStimuli[inds_animalSession]
            try: 
                dateOfExperiment = df_animalSession['date'].values[0]
            except:
                dateOfExperiment = '?'
            print('date of exp:', dateOfExperiment)
            
            # get lists of trial IDs matching stimuli
            trials_stimA, trials_stimB = getListsOfTrialIDs(df_animalSession)
            
            #### skip this comparison <-- if the data don't match in number of trials 
            numTimePtsPerTrial = getNumTimePtsPerTrial(df_animalSession,trials_stimA,trials_stimB)
            if areNumTrialsPerStimulusEqual(numTimePtsPerTrial)==False:
                break     
                
#             #### skip this comparison <-- if the data don't match in approx number of timePts
            if areNumTimePtsPerTrialSimilar(numTimePtsPerTrial) == False:
                break
            
            # animal, session 
            
            #### select time points
            minTPs, maxTPs, meanTPs, stdTPs = timePtStats(numTimePtsPerTrial)
            truncLst = []
            timingsMissing = False
            isFirstTrialExamined = True
            minStimulusDuration = []
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                
                ## get stimulus timings
                onsetFrame = getOnsetFrameNum(animal,session,trial)
                offsetFrame = getOffsetFrameNum(animal,session,trial)
                stimulusDuration = offsetFrame - onsetFrame
#                 print('stimulus duration',stimulusDuration)
#                 print('onset frame', onsetFrame)
#                 print('offset frame', offsetFrame)
                
                ## break out of both loops if timings are missing
                if np.isnan(onsetFrame) or np.isnan(offsetFrame):
                    print('stimulus timing missing --> skipping')
                    timingsMissing = True
                    break
                    
                ## set as min if first or has min duration
#                 print('min stim dur', minStimulusDuration)
#                 print('first trial examined', isFirstTrialExamined)
                if isFirstTrialExamined == True:
                    isFirstTrialExamined = False
                    minStimulusDuration = stimulusDuration[()]
                    print('first trial examined --> minStimDur is just the stimDur', minStimulusDuration)
                elif stimulusDuration[()] < minStimulusDuration and isFirstTrialExamined == False:
                    print('new shortest is: ', stimulusDuration)
                    minStimulusDuration = stimulusDuration[()]
                minStimulusDuration = int(minStimulusDuration)
                
            ## timings are missing --> skip the rest of the analysis for this set of data becausing 
            if timingsMissing == True: 
                break   
            
            #### truncate data, then select time points, and save to new df
            truncLst = []
            print('trial concat', np.concatenate((trials_stimA,trials_stimB)))
            for trial in np.concatenate((trials_stimA,trials_stimB)):
                onsetFrame = getOnsetFrameNum(animal,session,trial)
                offsetFrame_chosen = onsetFrame + minStimulusDuration
                print('min stim dur', minStimulusDuration)
                print('onset frame', getOnsetFrameNum(animal,session,trial))
                print('offsetFrame_chosen: ',offsetFrame_chosen)
                
                tmp_inds_trunc = (df_animalSession['trial']==trial) & (df_animalSession['timePt'] >= onsetFrame.astype(int)) & (df_animalSession['timePt'] < offsetFrame_chosen.astype(int))
                tmp_df_trunc = df_animalSession[tmp_inds_trunc]
                truncLst.append(tmp_df_trunc)
            df_trunc = pd.concat(truncLst)
            
            ### just an optional sanity check
            print('num time pts in df_trunc')
            getNumTimePtsPerTrial(df_trunc, trials_stimA, trials_stimB) ### work
            ###
            
            #### concatenate same cells 
            ### loop over and concatenate neurons into the same row if they're the same neuron and stimuli 
            ##      (ie mouse, session, stimuli)
            print(animal,session)
            neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB = sameNeuronConcat(df_trunc, minStimulusDuration)
            print('shape', np.shape(neuronArr_anmlSess_stimA), np.shape(neuronArr_anmlSess_stimB))
            
            ######### SVM #########
            
            ## create SVM format input by concatenating both classes (stimuli types); y is the labels
            print("stimA, stimB",np.shape(neuronArr_anmlSess_stimA), np.shape(neuronArr_anmlSess_stimB))
            X = np.concatenate((neuronArr_anmlSess_stimA, neuronArr_anmlSess_stimB), axis = 0)
            y = np.empty((neuronArr_anmlSess_stimA.shape[0]+neuronArr_anmlSess_stimB.shape[0]))
            y[:neuronArr_anmlSess_stimA.shape[0]] = 0
            y[neuronArr_anmlSess_stimB.shape[0]:] = 1
            print("X:", X.shape)
            print("y:", y.shape)
            
            print("k fold partitioning")
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
            
            print("training SVM")
            clf = svm.SVC(kernel='linear')
            
            ### WORK
#             if ind_comparison == 14:
#                 np.savetxt("y_train_works.csv", y_train, delimiter=",")
#                 np.savetxt("X_train_works.csv", X_train, delimiter=",")
            
            try:
                
                print(ind_comparison) 
                clf.fit(X_train.astype('float128'), y_train)
                print("testing SVM")
                tmp_score = clf.score(X_test.astype('float128'), y_test)
                print(tmp_score)
            except:
                print('X_train finite?: ', np.all(np.isfinite(X_train)))
                print('X_test finite?: ', np.all(np.isfinite(X_test)))
#                 np.savetxt("y_train_broken.csv", y_train, delimiter=",")
#                 np.savetxt("X_train_broken.csv", X_train, delimiter=",")
                pdb.set_trace()
            ### WORK
            
            ## add each SVM result to df
            analysisDate = pd.to_datetime('now')
            df_SVM.loc[ind_comparison] = [analysisDate, dateOfExperiment, animal, session, stimA, stimB, tmp_score]
#             print(df_SVM)

            ind_comparison += 1
            print('\n')
        print('########\n')
print('total number of comparisons: ', ind_comparison+1)

('USS', 'baseline')
stimuli comparison num:  1
animal:  1
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 5 10 16] [ 1  8 15]
stimuliType x presentationNum (row x col)
[[ 1117.  1116.  1117.]
 [  370.   363.  1476.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
stimulus timing missing --> skipping
########

stimuli comparison num:  1
animal:  3
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 4 11 17] [ 1  8 15]
stimuliType x presentationNum (row x col)
[[ 1267.  1127.  1123.]
 [  674.   682.   678.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
stimulus timing missing --> skipping
########

stimuli comparison num:  1
animal:  4
session: 1
date of exp: 2017_05_00
USS baseline
trial IDs for each stimulus type [ 4 11] []
stimuliType x presentationNum (row x col)
[[ 1137.  1113.]

stimulus: female
truncation frame:  110
appending same neurons in trial:  3
same stim: (35, 110)
appending same neurons in trial:  13
same stim: (35, 220)
appending same neurons in trial:  17
same stim: (35, 330)
shape (35, 330) (35, 330)
stimA, stimB (35, 330) (35, 330)
X: (70, 330)
y: (70,)
k fold partitioning
training SVM
4
testing SVM
0.714285714286


session: 3
date of exp: 2017_05_00
USS female
trial IDs for each stimulus type [ 3 14 17] [ 4 13 16]
stimuliType x presentationNum (row x col)
[[ 1019.  1087.  1156.]
 [ 1010.  1002.  1019.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
trial concat [ 3 14 17  4 13 16]
min stim dur 110
onset frame 341.0
offsetFrame_chosen:  451.0
min stim dur 110
onset frame 339.0
offsetFrame_chosen:  449.0
min stim dur 110
onset frame 340.0
offsetFrame_chosen:  450.0
min stim dur 110
onset frame 354.0
offsetFrame_chosen:  464.0
min stim dur 110
onset frame 330.0
offsetFrame_chose

trial concat [ 6 12 16  5 11 19]
min stim dur 110
onset frame 339.0
offsetFrame_chosen:  449.0
min stim dur 110
onset frame 338.0
offsetFrame_chosen:  448.0
min stim dur 110
onset frame 351.0
offsetFrame_chosen:  461.0
min stim dur 110
onset frame 349.0
offsetFrame_chosen:  459.0
min stim dur 110
onset frame 320.0
offsetFrame_chosen:  430.0
min stim dur 110
onset frame 361.0
offsetFrame_chosen:  471.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 110.  110.  110.]
 [ 110.  110.  110.]]
3 2

stimulus: USS
truncation frame:  110
appending same neurons in trial:  6
same stim: (35, 110)
appending same neurons in trial:  12
same stim: (35, 220)
appending same neurons in trial:  16
same stim: (35, 330)

stimulus: male
truncation frame:  110
appending same neurons in trial:  5
same stim: (35, 110)
appending same neurons in trial:  11
same stim: (35, 220)
appending same neurons in trial:  19
same stim: (35, 330)
shape (35, 330) (35, 330)
stimA, stimB (35, 330) (35, 330)

num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.  111.]
 [ 111.  111.]]
5 3

stimulus: USS
truncation frame:  111
appending same neurons in trial:  3
same stim: (20, 111)
appending same neurons in trial:  14
same stim: (20, 222)

stimulus: male
truncation frame:  111
appending same neurons in trial:  1
same stim: (20, 111)
appending same neurons in trial:  16
same stim: (20, 222)
shape (20, 222) (20, 222)
stimA, stimB (20, 222) (20, 222)
X: (40, 222)
y: (40,)
k fold partitioning
training SVM
17
testing SVM
0.5


########

stimuli comparison num:  19
animal:  7
session: 1
date of exp: 2017_05_00
USS male
trial IDs for each stimulus type [ 2 14] [ 7 10]
stimuliType x presentationNum (row x col)
[[ 1030.  1039.]
 [ 1040.  1018.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
trial concat [ 2 14  7 10]
min stim dur 111
onset frame 309.0
offsetFrame_chosen:  420.0
min stim dur 111
onset frame 30

same stim: (36, 111)
appending same neurons in trial:  9
same stim: (36, 222)

stimulus: mineral oil odor
truncation frame:  111
appending same neurons in trial:  4
same stim: (36, 111)
appending same neurons in trial:  15
same stim: (36, 222)
shape (36, 222) (36, 222)
stimA, stimB (36, 222) (36, 222)
X: (72, 222)
y: (72,)
k fold partitioning
training SVM
24
testing SVM
0.8


session: 2
date of exp: 2017_05_00
USS mineral oil odor
trial IDs for each stimulus type [3] [2]
stimuliType x presentationNum (row x col)
[[ 1251.]
 [ 1121.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 101.0
trial concat [3 2]
min stim dur 101
onset frame 342.0
offsetFrame_chosen:  443.0
min stim dur 101
onset frame 328.0
offsetFrame_chosen:  429.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 101.]
 [ 101.]]
5 2

stimulus: USS
truncation frame:  101
appending same neurons in trial:  3
same stim: (35, 101)

stimulus: miner

...

rat toy
trial IDs for each stimulus type [ 5 10 16] [ 3 12 19]
stimuliType x presentationNum (row x col)
[[ 1120.  1114.  1322.]
 [ 1131.  1114.  1113.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 133.0
new shortest is:  114.0
new shortest is:  111.0
trial concat [ 5 10 16  3 12 19]
min stim dur 111
onset frame 376.0
offsetFrame_chosen:  487.0
min stim dur 111
onset frame 397.0
offsetFrame_chosen:  508.0
min stim dur 111
onset frame 443.0
offsetFrame_chosen:  554.0
min stim dur 111
onset frame 397.0
offsetFrame_chosen:  508.0
min stim dur 111
onset frame 296.0
offsetFrame_chosen:  407.0
min stim dur 111
onset frame 353.0
offsetFrame_chosen:  464.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.  111.  111.]
 [ 111.  111.  111.]]
1 2

stimulus: rat
truncation frame:  111
appending same neurons in trial:  5
same stim: (25, 111)
appending same neurons in trial:  10
same stim: (25, 222)
appending

stimuliType x presentationNum (row x col)
[[ 1131.  1146.]
 [ 1120.  1122.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
trial concat [ 8 14  2 15]
min stim dur 111
onset frame 430.0
offsetFrame_chosen:  541.0
min stim dur 111
onset frame 345.0
offsetFrame_chosen:  456.0
min stim dur 111
onset frame 339.0
offsetFrame_chosen:  450.0
min stim dur 111
onset frame 331.0
offsetFrame_chosen:  442.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.  111.]
 [ 111.  111.]]
4 2

stimulus: rat
truncation frame:  111
appending same neurons in trial:  8
same stim: (49, 111)
appending same neurons in trial:  14
same stim: (49, 222)

stimulus: toy
truncation frame:  111
appending same neurons in trial:  2
same stim: (49, 111)
appending same neurons in trial:  15
same stim: (49, 222)
shape (49, 222) (49, 222)
stimA, stimB (49, 222) (49, 222)
X: (98, 222)
y: (98,)
k fold partitioning
training SVM
291
testi

tone toy
trial IDs for each stimulus type [ 6 14 17] [ 3 12 19]
stimuliType x presentationNum (row x col)
[[ 1137.  1113.  1127.]
 [ 1131.  1114.  1113.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 110.0
trial concat [ 6 14 17  3 12 19]
min stim dur 110
onset frame 339.0
offsetFrame_chosen:  449.0
min stim dur 110
onset frame 338.0
offsetFrame_chosen:  448.0
min stim dur 110
onset frame 340.0
offsetFrame_chosen:  450.0
min stim dur 110
onset frame 397.0
offsetFrame_chosen:  507.0
min stim dur 110
onset frame 296.0
offsetFrame_chosen:  406.0
min stim dur 110
onset frame 353.0
offsetFrame_chosen:  463.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 110.  110.  110.]
 [ 110.  110.  110.]]
1 2

stimulus: tone
truncation frame:  110
appending same neurons in trial:  6
same stim: (25, 110)
appending same neurons in trial:  14
same stim: (25, 220)
appending same neurons in trial:  17
same stim: (25, 33

stimuliType x presentationNum (row x col)
[[ 1115.  1121.]
 [ 1120.  1122.]]
trial numbers match
checking approx num of time points
first trial examined --> minStimDur is just the stimDur 111.0
trial concat [ 7 12  2 15]
min stim dur 111
onset frame 331.0
offsetFrame_chosen:  442.0
min stim dur 111
onset frame 328.0
offsetFrame_chosen:  439.0
min stim dur 111
onset frame 339.0
offsetFrame_chosen:  450.0
min stim dur 111
onset frame 331.0
offsetFrame_chosen:  442.0
num time pts in df_trunc
stimuliType x presentationNum (row x col)
[[ 111.  111.]
 [ 111.  111.]]
4 2

stimulus: tone
truncation frame:  111
appending same neurons in trial:  7
same stim: (49, 111)
appending same neurons in trial:  12
same stim: (49, 222)

stimulus: toy
truncation frame:  111
appending same neurons in trial:  2
same stim: (49, 111)
appending same neurons in trial:  15
same stim: (49, 222)
shape (49, 222) (49, 222)
stimA, stimB (49, 222) (49, 222)
X: (98, 222)
y: (98,)
k fold partitioning
training SVM
304
test

same stim: (54, 222)

stimulus: toy
truncation frame:  111
appending same neurons in trial:  5
same stim: (54, 111)
appending same neurons in trial:  15
same stim: (54, 222)
shape (54, 222) (54, 222)
stimA, stimB (54, 222) (54, 222)
X: (108, 222)
y: (108,)
k fold partitioning
training SVM
311
testing SVM
0.772727272727


########

total number of comparisons:  313


In [7]:
## buffer cell
pdb.set_trace()

np.savetxt("X_train.csv", X_train, delimiter=",")

            #             tmp_SVMresult = pd.DataFrame({"animal": [animal]})#,{"testAccuracy": tmp_score})
#             tmp_SVMresult = pd.DataFrame([animal], columns = list([1])) #,{"testAccuracy": tmp_score})
#             df_SVM.loc[df_SVM.index.max() + 1] = [animal, session, tmp_score]
#             df_SVM.loc[totalNumComparisons] = [animal, session, tmp_score]
#             df_SVM.loc[totalNumComparisons] = [0,1,2]
#             df_SVM = pd.DataFrame({'mouse': animal, 'session': session, 'SVMaccuracy': tmp_score}, index =totalNumComparisons)
#             df_SVM.iloc[1] = dict(x=9, y=99)
#             df_SVM.append(tmp_SVMresult, ignore_index=True)
#             clf.predict(X_test, y_test)
            
    
    ##########33
#             cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2, random_state=0)
            
            # WORK: optional gridsearch
#             gammas = np.logspace(-6, -1, 10)
#             classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=dict(gamma=gammas))
#             classifier.fit(X_train, y_train)
            
#             title = 'Learning Curves (SVM, linear kernel, $\gamma=%.6f$)' %classifier.best_estimator_.gamma
#             estimator = SVC(kernel='linear', gamma=classifier.best_estimator_.gamma)
#             plot_learning_curve(estimator, title, X_train, y_train, cv=cv)
#             plt.show()




--Return--
> <ipython-input-7-96dded62fc71>(2)<module>()->None
-> pdb.set_trace()
(Pdb) q


BdbQuit: 

In [ ]:
## export timings
df_timings.to_csv('df_timings.csv', header=True, index=True)

In [25]:
## save raw svm (animal/session) results to /gitTracked/python/
timestr = time.strftime("%Y_%m_%d__%H_%M")
print(timestr)

# Write out raw analysis to csv file
OUTPUT_FILENAME = 'SVM_analysis_raw' + timestr + '.csv'
df_SVM.to_csv(OUTPUT_FILENAME, header=True, index=False)

2017_07_27__01_21


In [26]:
## save summary svm data

# svmGrouped, df_SVM_summary = df_SVM.groupby(['stimulusA', 'stimulusB'], as_index=False)

# f = {'SVM_accuracy':['sum','mean'], 'B':['prod']}

stimGrouped = df_SVM.groupby(['stimulusA','stimulusB'], as_index=True)
df_SVM_summaryDesc = stimGrouped.describe()

print(stimGrouped.describe())      

# Write out raw analysis to csv file
OUTPUT_FILENAME = 'SVM_analysis_summary' + timestr + '.csv'
df_SVM_summaryDesc.to_csv(OUTPUT_FILENAME, header=True, index=True)

                                  SVM_accuracy                                \
                                         count      mean       std       min   
stimulusA        stimulusB                                                     
USS              female                    6.0  0.661376  0.119112  0.444444   
                 male                     15.0  0.751247  0.146639  0.500000   
                 mineral oil odor          9.0  0.799023  0.134232  0.500000   
                 peanut odor               7.0  0.794965  0.047757  0.727273   
                 pred odor                 7.0  0.727979  0.137724  0.533333   
                 rat                      13.0  0.735339  0.173174  0.333333   
                 tone                     15.0  0.638125  0.198350  0.200000   
                 toy                      15.0  0.632092  0.190718  0.214286   
female           male                      6.0  0.699206  0.191223  0.333333   
                 rat                    

In [15]:
## junk / notes

ratData = df.loc[df['stimulusType'] == 'rat',:]
ussData = df.loc[df['stimulusType'] == 'USS',:]

print(ussData)



df.corr()


# Rename the impact force column
df = df.rename(columns={'impact force (mN)': 'impf'})



KeyError: 'stimulusType'